In [1]:
from utils import *
from run_tracking import *
import napari
import os

In [2]:
# path to movie, folder for raw images and folder for edited images
path_movie = 'test/movie20191011_094113.mp4'
folder_out_raw = 'test/raw'
folder_out_rem_backgr = 'test/rem_backgr'

for folder in [folder_out_raw, folder_out_rem_backgr]:
    if not os.path.exists(folder):
        os.mkdir(folder)

In [ ]:
# convert video to images and save as tiff files
mp4toimgs(path_movie, folder_out_raw)

# remove background and save as gray images
remove_background(path_movie, folder_out_rem_backgr)

In [6]:
# delete the first few images (because background subtraction is bad)
n_del = 5
imgs_raw = [file for file in os.listdir(folder_out_raw) if file.endswith(".tif")]
imgs_rmb = [file for file in os.listdir(folder_out_rem_backgr) if file.endswith(".tif")]

for i in range(n_del):
    os.remove(folder_out_raw + '/' + imgs_raw[i])
    os.remove(folder_out_rem_backgr + '/' + imgs_rmb[i])

In [3]:
# detect
all_detections = detect(folder_out_rem_backgr)
det_napari = detections_napari(all_detections)

# track
tracks, trackabilities, _, _ = track(all_detections, observation_noise=1, transition_noise=10, trackability=False)
trs_napari = tracks_napari(tracks)

100%|██████████| 5/5 [00:03<00:00,  1.26it/s]


In [ ]:
# display in napari
movie = folder_tifs_to_single_tif(folder_out_raw)
viewer = napari.Viewer()
#viewer.add_image(movie)
viewer.add_points(np.array(det_napari), size=10, face_color='red', name='detections')
viewer.add_tracks(np.array(trs_napari), name='tracks')
napari.run()